In [1]:
import re
import sys
import warnings
from collections import Counter, defaultdict
from federated_eval_helper_functions import *
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.stats.api as sms
from matplotlib import pyplot as plt
from pylab import cm
from scipy.stats import ttest_ind
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import statistics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    LabelBinarizer,
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)

In [2]:
print(sys.executable)
print(sys.version)
print(sys.version_info)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/bin/python
3.9.10 | packaged by conda-forge | (main, Jan 30 2022, 18:02:27) 
[Clang 11.1.0 ]
sys.version_info(major=3, minor=9, micro=10, releaselevel='final', serial=0)


# Load Data

In [3]:
silo1 = pd.read_csv(
    "/Users/joaoalmeida/Desktop/tese_local/chvng/CHVNGE_OBSCARE_WCR.csv", sep=";"
)

In [4]:
silo2 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHEDV/CHEDV_OBSCARE_WCR_.xlsx",
    engine="openpyxl",
)

In [5]:
silo3 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHBV/CHBV_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo4 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHLN/CHLN_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo5 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHSJ/CHSJ_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo6 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/HSO/HSO_OBSCARE_WCR.xlsx",
    engine="openpyxl",
)

silo7 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/ULSAM/ULSAM_OBSCARE_WCR.xls"
)

silo8 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/ULSM/ULSM_OBSCARE_WCR_V1.xlsx",
    engine="openpyxl",
)

silo9 = pd.read_excel(
    "/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/CHTS/CHTS_OBSCARE_WCR_.xlsx",
    engine="openpyxl",
)

WARNING *** file size (5079264) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


# Prepare data

In [6]:
silos = []
for i in range(9):
    silos.append(eval("silo" + str(i + 1)))

In [7]:
silo4.describe(include="all")

,IDENTIFICADOR,DATA_PARTO,IDADE_MATERNA,GS,PESO_INICIAL,PESO_ADMISSAO_INTERNAMENTO,IMC,CIGARROS,ALCOOL,ESTUPEFACIENTES,...,DIABETES_TIPO2,HEMATOLOGICA,RESPIRATORIA,CEREBRAL,CARDIACA,TIPO_PARTO,APRESENTACAO_NO_PARTO,TRAB_PARTO_NO_PARTO,SEMANAS_GESTACAO_PARTO,GRUPO_ROBSON
count,2364.000000,2364,2364.000000,2364,2224.000000,88.000000,2217.000000,243,33,6,...,1,154,129,34,211,2364,2364,2364,2364.000000,623
unique,NaN,409,NaN,12,NaN,NaN,NaN,1,1,1,...,1,1,1,1,1,7,5,3,NaN,10
top,NaN,22/09/2020,NaN,"O,RH_POSITIVO",NaN,NaN,NaN,SIM,SIM,SIM,...,X,SIM,SIM,SIM,SIM,Parto eutócico cefálico,Cefálica de vértice,Espontâneo,NaN,1
freq,NaN,12,NaN,896,NaN,NaN,NaN,243,33,6,...,1,154,129,34,211,1233,2257,1460,NaN,163
mean,8797.050338,NaN,31.096870,NaN,65.524775,79.601136,24.961344,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.913283,NaN
std,1052.560060,NaN,6.288548,NaN,14.114238,16.338330,8.894069,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.943514,NaN
min,5573.000000,NaN,15.000000,NaN,20.000000,36.600000,7.800000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN
25%,7923.500000,NaN,27.000000,NaN,56.000000,70.000000,21.100000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.400000,NaN
50%,8789.000000,NaN,31.000000,NaN,63.000000,77.250000,23.700000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.400000,NaN
75%,9640.500000,NaN,36.000000,NaN,73.000000,88.625000,27.300000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.300000,NaN


In [8]:
int_cols = [
    "IDADE_MATERNA",
    "PESO_INICIAL",
    "IMC",
    "NUMERO_CONSULTAS_PRE_NATAL",
    "IDADE_GESTACIONAL_ADMISSAO",
    "SEMANAS_GESTACAO_PARTO",
    "PESO_ADMISSAO_INTERNAMENTO"


]
%store int_cols

Stored 'int_cols' (list)


In [9]:
drop_cols = [
    "IDENTIFICADOR",
    "G_TERAPEUTICA",
    "DATA_PARTO",
    "CESARIANAS_MOTIVO_ANTERIOR",
    "NUM_RN",
    "E_ALT_UT",
      'HIPERTENSAO_CRONICA',
         'HIPERTENSAO_GESTACIONAL',
         'HIPERTENSAO_PRE_ECLAMPSIA',
         'DIABETES_GESTACIONAL_DIETA',
         'DIABETES_GESTACIONAL_INSULINA',
         'DIABETES_GESTACIONAL_ANTIBIO',
         'DIABETES_MATERNA',
         'DIABETES_TIPO1',
         'DIABETES_TIPO2',
         'HEMATOLOGICA',
         'RESPIRATORIA',
         'CEREBRAL',
         'CARDIACA',
    "ESTIMATIVA_PESO_ECO_24",
    "ESTIMATIVA_PESO_ECO_25",
    "ESTIMATIVA_PESO_ECO_26",
    "ESTIMATIVA_PESO_ECO_27",
    "ESTIMATIVA_PESO_ECO_28",
    "ESTIMATIVA_PESO_ECO_29",
    "ESTIMATIVA_PESO_ECO_30",
    "ESTIMATIVA_PESO_ECO_31",
    "ESTIMATIVA_PESO_ECO_32",
    "ESTIMATIVA_PESO_ECO_33",
    "ESTIMATIVA_PESO_ECO_34",
    "ESTIMATIVA_PESO_ECO_35",
    "ESTIMATIVA_PESO_ECO_36",
    "ESTIMATIVA_PESO_ECO_37",
    "ESTIMATIVA_PESO_ECO_38",
    "ESTIMATIVA_PESO_ECO_39",
    "ESTIMATIVA_PESO_ECO_40",
    "ESTIMATIVA_PESO_ECO_41",
    "ESTIMATIVA_PESO_ECO_42",
    "APRESENTACAO_24",
    "APRESENTACAO_25",
    "APRESENTACAO_26",
    "APRESENTACAO_27",
    "APRESENTACAO_28",
    "APRESENTACAO_29",
    "APRESENTACAO_30",
    "APRESENTACAO_31",
    "APRESENTACAO_32",
    "APRESENTACAO_33",
    "APRESENTACAO_34",
    "APRESENTACAO_35",
    "APRESENTACAO_36",
    "APRESENTACAO_37",
    "APRESENTACAO_38",
    "APRESENTACAO_39",
    "APRESENTACAO_40",
    "APRESENTACAO_41",
    "APRESENTACAO_42",
"ALCOOL",
"ESTUPEFACIENTES",
    "VIGIADA_HOSPITAL"
]

In [10]:
cat_cols = list(set(silos[0].columns) - set(drop_cols) - set(int_cols))
%store cat_cols

Stored 'cat_cols' (list)


In [11]:
silos_null=[]
for silo in silos:
    
    silos_null.extend(silo.columns[silo.isna().all()].tolist())
Counter(silos_null) 

Counter({'CESARIANAS_MOTIVO_ANTERIOR': 8,
         'ESTIMATIVA_PESO_ECO_41': 7,
         'ESTIMATIVA_PESO_ECO_42': 9,
         'APRESENTACAO_42': 9,
         'APRESENTACAO_41': 7,
         'E_ALT_UT': 3,
         'ESTIMATIVA_PESO_ECO_24': 1,
         'ESTIMATIVA_PESO_ECO_25': 1,
         'ESTIMATIVA_PESO_ECO_26': 1,
         'ESTIMATIVA_PESO_ECO_27': 1,
         'ESTIMATIVA_PESO_ECO_28': 1,
         'ESTIMATIVA_PESO_ECO_29': 1,
         'ESTIMATIVA_PESO_ECO_30': 1,
         'ESTIMATIVA_PESO_ECO_31': 1,
         'ESTIMATIVA_PESO_ECO_32': 1,
         'ESTIMATIVA_PESO_ECO_33': 1,
         'ESTIMATIVA_PESO_ECO_34': 1,
         'ESTIMATIVA_PESO_ECO_35': 1,
         'ESTIMATIVA_PESO_ECO_36': 1,
         'ESTIMATIVA_PESO_ECO_37': 1,
         'ESTIMATIVA_PESO_ECO_38': 1,
         'ESTIMATIVA_PESO_ECO_39': 1,
         'ESTIMATIVA_PESO_ECO_40': 1,
         'APRESENTACAO_40': 1,
         'APRESENTACAO_39': 1,
         'APRESENTACAO_38': 1,
         'APRESENTACAO_37': 1,
         'APRESENTACAO_36'

In [12]:
high_null=[]
for silo in silos:
         for col in silo.columns:
            if sum(silo[col].isna())/len(silo)>0.9:
                high_null.append(col)
for k,v in Counter(high_null).items():
    if v>8:
        print(k)
        if k not in drop_cols:
            drop_cols.append(k)

ALCOOL
ESTUPEFACIENTES
CESARIANAS_MOTIVO_ANTERIOR
VIGIADA_HOSPITAL
ESTIMATIVA_PESO_ECO_24
ESTIMATIVA_PESO_ECO_25
ESTIMATIVA_PESO_ECO_26
ESTIMATIVA_PESO_ECO_27
ESTIMATIVA_PESO_ECO_38
ESTIMATIVA_PESO_ECO_39
ESTIMATIVA_PESO_ECO_40
ESTIMATIVA_PESO_ECO_41
ESTIMATIVA_PESO_ECO_42
APRESENTACAO_42
APRESENTACAO_41
APRESENTACAO_40
APRESENTACAO_39
APRESENTACAO_38
APRESENTACAO_27
APRESENTACAO_26
APRESENTACAO_25
APRESENTACAO_24
E_ALT_UT
HIPERTENSAO_CRONICA
HIPERTENSAO_GESTACIONAL
HIPERTENSAO_PRE_ECLAMPSIA
DIABETES_GESTACIONAL_DIETA
DIABETES_GESTACIONAL_INSULINA
DIABETES_GESTACIONAL_ANTIBIO
DIABETES_MATERNA
DIABETES_TIPO1
DIABETES_TIPO2
HEMATOLOGICA
RESPIRATORIA
CEREBRAL
CARDIACA


In [13]:
# get null count
null_counts_dict={}
for col in silos[0].columns:
    null_counts_dict[col]=[]
    for silo in silos:
        null_counts_dict[col].append(round(sum(silo[col].isna()/len(silo)),3))

In [14]:
nulls_df=pd.DataFrame.from_dict(null_counts_dict).T

In [15]:

nulls_df['mean'] = nulls_df.mean(axis=1)
#""" targets=[      "GRUPO_ROBSON OK",
 #       "TIPO_GRAVIDEZ OK",
  #      "EUTOCITO_ANTERIOR OK",
   #     "CESARIANAS_ANTERIOR",
   #     "TRAB_PARTO_NO_PARTO OK",
   #     "TIPO_PARTO OK",
   #     "BACIA",
   #     "DIABETES_GESTACIONAL",
   #     "GS"] OK,"""

nulls_df.loc[["GS","TIPO_GRAVIDEZ","GRUPO_ROBSON","EUTOCITO_ANTERIOR","CESARIANAS_ANTERIOR","TRAB_PARTO_NO_PARTO","TIPO_PARTO","BACIA","DIABETES_GESTACIONAL"],:]

,0,1,2,3,4,5,6,7,8,mean
GS,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000
TIPO_GRAVIDEZ,0.311,0.088,0.129,0.315,0.073,0.090,0.125,0.118,0.060,0.145444
GRUPO_ROBSON,0.000,0.057,0.002,0.736,0.001,0.001,0.001,0.003,0.002,0.089222
EUTOCITO_ANTERIOR,0.747,0.603,0.749,0.673,0.454,0.588,0.612,0.689,0.615,0.636667
CESARIANAS_ANTERIOR,0.879,0.726,0.861,0.895,0.621,0.691,0.745,0.855,0.778,0.783444
TRAB_PARTO_NO_PARTO,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.111111
TIPO_PARTO,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000
BACIA,0.954,0.777,0.901,0.969,0.812,0.896,0.529,0.931,0.815,0.842667
DIABETES_GESTACIONAL,0.877,0.900,0.902,0.908,0.898,0.922,0.882,0.899,0.868,0.895111


In [16]:
nulls_df[nulls_df["mean"]<0.5]

#CAT:


#GS
#A_PARA
#A_GESTA
#TIPO_GRAVIDEZ
#VIGIADA
#VIGIADA_CENTRO_SAUDE
#VIGIADA_NESTE_HOSPITAL
#APRESENTACAO_ADMISSAO
#TRAB_PARTO_ENTRADA_ESPONTANEO
#TIPO_PARTO
#APRESENTACAO_NO_PARTO
#TRAB_PARTO_NO_PARTO
#GRUPO_ROBSON


#NUMERIC

#IDADE_MATERNA
#PESO_INICIAL
#IMC
#NUMERO_CONSULTAS_PRE_NATAL
#IDADE_GESTACIONAL_ADMISSAO
#SEMANAS_GESTACAO_PARTO

,0,1,2,3,4,5,6,7,8,mean
IDENTIFICADOR,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000
DATA_PARTO,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000
IDADE_MATERNA,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000111
GS,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000000
PESO_INICIAL,0.069,0.167,0.040,0.059,0.059,0.408,0.045,0.166,0.084,0.121889
IMC,0.075,0.209,0.054,0.062,0.070,0.417,0.056,0.173,0.104,0.135556
A_PARA,0.448,0.414,0.275,0.420,0.327,0.381,0.426,0.412,0.432,0.392778
A_GESTA,0.000,0.000,0.002,0.000,0.000,0.000,0.000,0.001,0.000,0.000333
TIPO_GRAVIDEZ,0.311,0.088,0.129,0.315,0.073,0.090,0.125,0.118,0.060,0.145444
VIGIADA,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.001,0.000,0.000333


In [17]:
nulls_df[nulls_df["mean"]>0.9]

,0,1,2,3,4,5,6,7,8,mean
ALCOOL,0.991,0.973,0.988,0.986,0.993,0.983,0.992,0.983,0.987,0.986222
ESTUPEFACIENTES,0.996,0.998,0.999,0.997,0.998,0.999,0.998,0.998,0.998,0.997889
CESARIANAS_MOTIVO_ANTERIOR,1.000,1.000,1.000,1.000,0.996,1.000,1.000,1.000,1.000,0.999556
VIGIADA_HOSPITAL,0.957,0.975,0.976,0.927,0.933,0.990,0.979,0.944,0.980,0.962333
ESTIMATIVA_PESO_ECO_24,0.921,0.944,0.973,1.000,0.958,0.992,0.987,0.980,0.991,0.971778
ESTIMATIVA_PESO_ECO_25,0.950,0.980,0.973,1.000,0.977,0.991,0.991,0.981,0.975,0.979778
ESTIMATIVA_PESO_ECO_26,0.967,0.968,0.985,1.000,0.951,0.986,0.988,0.979,0.972,0.977333
ESTIMATIVA_PESO_ECO_27,0.957,0.966,0.982,1.000,0.975,0.989,0.987,0.973,0.963,0.976889
ESTIMATIVA_PESO_ECO_28,0.879,0.861,0.918,1.000,0.912,0.985,0.981,0.910,0.894,0.926667
ESTIMATIVA_PESO_ECO_29,0.924,0.886,0.962,1.000,0.952,0.979,0.985,0.964,0.961,0.957000


In [18]:
# get null count
null_counts_dict={}
for idx,silo in enumerate(silos):
    null_counts_dict["silo"+str(idx+1)]={}
    for col in silo.columns:
        null_counts_dict["silo"+str(idx+1)][col]=(sum(silo[col].isna()/len(silo)))

In [19]:
def create_pipeline_with_y(
    data,
    target,
    int_cols,
    cat_cols,
    manual_encoder,
    drop_cols=None,
    pipeline=None,
    op="encoder",
    as_df=False,
):
    _cat_cols = [colc for colc in cat_cols if colc not in [target]]
    _int_cols = [coli for coli in int_cols if coli not in [target]]
    n_df = data.copy().drop(columns=drop_cols + [target])

    def mapping_enc(x,encoder):
      #  print(x,encoder)
        return encoder[x]
    
    vfunc = np.vectorize(mapping_enc)

    def manual_ordinal_encoder(x):
        for idx,c in enumerate(_cat_cols):
          #  print(manual_encoder)
          #  print(manual_encoder[c])
           # print(idx)
          #  print(x[:,idx])
          #  print("c",c)
          #  print(len(x[:,idx]))
            x[:,idx]=vfunc(x[:,idx],manual_encoder[c])
            print("passed")
        return x

    def to_object(x):
        return pd.DataFrame(x).astype(str)

    def to_number(x):
        return pd.DataFrame(x).astype(float)

    if pipeline == None:
        fun_str = FunctionTransformer(to_object)
        fun_num = FunctionTransformer(to_number)
        unified_ordinal_enconder = FunctionTransformer(manual_ordinal_encoder)

        numeric_transformer = Pipeline(
            steps=[                ("fun_num", fun_num),
                ("imputer1", SimpleImputer(strategy="median", missing_values=np.nan)),
                ("imputer2", SimpleImputer(strategy="median", missing_values=-1)),
            ]
        )

        categorical_transformer = Pipeline(
            steps=[
             #   ("fun_str", fun_str),
                (
                    "imputer",
                    SimpleImputer(
                        missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
                    ),
                ),
                ("ordinalEncoder", unified_ordinal_enconder),
            ]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, _int_cols),
                ("cat", categorical_transformer, _cat_cols),
            ]
        )

        pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

        y = data[target]
        X = n_df
        y_pipe = make_pipeline(
            SimpleImputer(
                missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
            )
        )
        yy, l = create_target(y, op=op)
        XX = pipeline.fit_transform(X)

        if as_df == True:
            col_list = (
                pipeline["preprocessor"].transformers_[0][2] #changes col location
                + pipeline["preprocessor"].transformers_[1][2]
            )
            print(len(col_list))
            df1 = pd.DataFrame(XX, columns=col_list)
            r = pd.concat([df1, pd.DataFrame(yy)], axis=1)
            r.rename(columns={0: target}, inplace=True)
            return r, None, pipeline, l
        return XX, yy, pipeline, l

In [20]:
def preprocessing_df(
    df,
    categorical_columns,
    integ_columns,
    drop_cols=["G_TERAPEUTICA", "IDENTIFICADOR"],
    pipeline=["imputer", "encoder"],
):
    if len(df.columns) != len(categorical_columns) + len(integ_columns) + len(drop_cols):
        # raise Exception
        raise("WARNING! Columns number different")
    df1 = df.copy()

    def simple_imputer_categorical_df(df, columns):
        imputer = SimpleImputer(
            missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
        )  # está a por NULL em variaveis com nrs(apesar de considerar cat)
        imputer.fit(df[columns])
        X = pd.DataFrame(imputer.transform(df[columns]))
        return X

    def simple_imputer_numeric_df(df, columns):
        imputer = SimpleImputer(missing_values=np.nan, strategy="median")
        imputer.fit(df[columns].astype(float))
        X = pd.DataFrame(imputer.transform(df[columns]))
        return X

    def scikit_one_hot_encoder(df, categorical_columns):
        enc_hot = OneHotEncoder(handle_unknown="ignore", sparse=False)
        enc_hot.fit(df[categorical_columns].astype(str))
        X = pd.DataFrame(
            enc_hot.transform(df[categorical_columns].astype(str)),
            columns=enc_hot.get_feature_names(df[categorical_columns].columns),
        )
        data_encoded = pd.concat([df, X], axis=1)
        data_encoded.drop(columns=df[categorical_columns].columns, inplace=True)
        return data_encoded

    df1.drop(columns=drop_cols, inplace=True)
    if "imputer" in pipeline:
        df1[categorical_columns] = simple_imputer_categorical_df(
            df1, categorical_columns
        )
        df1[integ_columns] = simple_imputer_numeric_df(df1, integ_columns)

    if "encoder" in pipeline:
        df1 = scikit_one_hot_encoder(df1, categorical_columns)

    return df1

In [21]:
import pickle
#transform all from SIM/NAO to S/N e similar
unique_values={}
standard_map={"ADEQUADA":"A","LIMITE":"L","DESCONHECIDO,RH_DESCONHECIDO":np.nan,"DESCONHECIDO,":np.nan,"SIM":"S","NAO":"N","-1":np.nan,-1:np.nan,"NS":np.nan,",":np.nan,"Sim":"S","UNKNOWN":np.nan,"Desconhecida":np.nan,"Desconhecido":np.nan,"DESCONHECIDO":np.nan," ":np.nan,"Desconhecido,":np.nan,"  ":np.nan}
def standardize_null(x,mapping):
    if x in mapping.keys():
        return mapping[x]
    if pd.isna(x):
        return np.nan
    return x


for silo in silos:
    for col in silo.columns:
        silo[col]=silo[col].apply(standardize_null,mapping=standard_map)
        
    for col in silo.columns:
        if col in cat_cols:
            if col in unique_values.keys():
                unique_values[col].extend(silo[col].unique())
            else:
                unique_values[col]=list(silo[col].unique())
for k,v in unique_values.items():
    unique_values[k]=["NULLIMP" if  pd.isnull(x) else x for x in unique_values[k]]
    unique_values[k]=set(unique_values[k])
manual_encoder={}
for k,v in unique_values.items():
    n=len(v)
    manual_encoder[k]=dict(zip(v,[str(x) for x in (range(0,n))]))
manual_encoder
#with open('manual_encoder.json', 'w') as outfile:
# write
with open('manual_encoder.pkl', 'wb') as f:
    pickle.dump(manual_encoder, f)

In [22]:
silosprep = []
silo_imputed = []
silos_stats = []
for silo in silos:
    
    silos_stats.append(silo.copy())
    silosprep.append(
        preprocessing_df(
            silo,
            categorical_columns=cat_cols,
            integ_columns=int_cols,
            drop_cols=drop_cols,
        )
    )
    silo_imputed.append(
        create_pipeline_with_y(
            silo.copy(),
            "GRUPO_ROBSON",
            int_cols=int_cols,
            manual_encoder=manual_encoder,
            cat_cols=cat_cols,
            as_df=True,
            drop_cols=drop_cols, 
        )[0]
    )

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


...

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
passed
35


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
resulting_list = []
for silo in silosprep:
    resulting_list.extend(x for x in silo.columns.tolist() if x not in resulting_list)
for col in resulting_list:
    for i in range(len(silosprep)):
        if col not in silosprep[i].columns:
            silosprep[i][col] = 0

In [24]:
c = Counter()
final_list = []
for s in silos_stats:
    c.update(s.columns)
for col in c.keys():
    if c[col] == 9:
        final_list.append(col)

In [25]:
for col in final_list:
    for i in range(len(silos_stats)):
        #  print(silos_stats[i].columns)
        if col not in silos_stats[i].columns:
            silos_stats[i][col] = np.nan

In [26]:
for silo in silos_stats:
    print(len(silo.columns))

96
96
96
96
96
96
96
96
96


In [27]:
pd.isnull(silos_stats[0]["BACIA"].mode(dropna=False).values[0])


True

In [28]:
silos_stats[0]["BACIA"].value_counts(dropna=False)[silos_stats[0]["BACIA"].mode(dropna=False).values[0]]

7671

In [29]:
def get_summary2(silo, col, type_col):
    if type_col == "categorical":
        if silo[col].isna().sum() == len(silo):
            return str("Null \textbf{100\%}")
        mode = silo[col].mode(dropna=False)
        freq = silo[col].value_counts(dropna=False)[mode].values[0]
        perc = freq / len(silo[col])
        print(mode)
        print(mode.values[0])
        if pd.isnull(mode.values[0]):
            return str("Null \textbf{"+str(int(round(perc * 100, 0))) + "\%}")
        if len(str(mode.values[0])) > 6:
            # return str(mode.values[0])[0:5].lower() +" .. "+str(mode.values[0])[-3:].lower()+ " [" + str(round(perc * 100, 1)) + "%]"
            return (
                str(mode.values[0])[0:4].lower()
                + ".. \textbf{"
                + str(int(round(perc * 100, 0)))
                + "\%}"
            )

        else:
            return str(mode.values[0]).lower() + " \textbf{" + str(int(round(perc * 100, 0))) + "\%}"
    # return " [" + str(round(perc * 100, 1)) + "%]"

    if type_col == "continuous":
        mean = silo[col].mean()
        std = silo[col].std()
        return str(round(mean, 1)) + " \textbf{" + str(round(std, 1)) + "}"

In [30]:
def create_analysis_silos(silos, cat_cols, int_cols, drop_cols):

    result = {}
    for idx, silo in enumerate(silos):
        silo = silo.copy().drop(columns=drop_cols)
        col_list = silo.columns.tolist()
        col_list.append("N (total)")
        result["silo" + str(idx + 1)] = []
        for col in silo.columns:
            #  print(col)
            type_col = "categorical" if col in cat_cols else "continuous"
            result["silo" + str(idx + 1)].append(get_summary2(silo, col, type_col))
        result["silo" + str(idx + 1)].append(str(len(silo)))

    return result, col_list

In [31]:
for k,v in Counter(high_null).items():
    if v>8:
        print(k)
        if k not in drop_cols:
            drop_cols.append(k)
cat_cols = list(set(silos[0].columns) - set(drop_cols) - set(int_cols))

ALCOOL
ESTUPEFACIENTES
CESARIANAS_MOTIVO_ANTERIOR
VIGIADA_HOSPITAL
ESTIMATIVA_PESO_ECO_24
ESTIMATIVA_PESO_ECO_25
ESTIMATIVA_PESO_ECO_26
ESTIMATIVA_PESO_ECO_27
ESTIMATIVA_PESO_ECO_38
ESTIMATIVA_PESO_ECO_39
ESTIMATIVA_PESO_ECO_40
ESTIMATIVA_PESO_ECO_41
ESTIMATIVA_PESO_ECO_42
APRESENTACAO_42
APRESENTACAO_41
APRESENTACAO_40
APRESENTACAO_39
APRESENTACAO_38
APRESENTACAO_27
APRESENTACAO_26
APRESENTACAO_25
APRESENTACAO_24
E_ALT_UT
HIPERTENSAO_CRONICA
HIPERTENSAO_GESTACIONAL
HIPERTENSAO_PRE_ECLAMPSIA
DIABETES_GESTACIONAL_DIETA
DIABETES_GESTACIONAL_INSULINA
DIABETES_GESTACIONAL_ANTIBIO
DIABETES_MATERNA
DIABETES_TIPO1
DIABETES_TIPO2
HEMATOLOGICA
RESPIRATORIA
CEREBRAL
CARDIACA


In [32]:
df_dict, col = create_analysis_silos(silos_stats, cat_cols, int_cols, drop_cols)

0    A,RH_POSITIVO
Name: GS, dtype: object
A,RH_POSITIVO
0    NaN
Name: CIGARROS, dtype: object
nan
0   NaN
Name: A_PARA, dtype: float64
nan
0    1
Name: A_GESTA, dtype: int64
1
0   NaN
Name: EUTOCITO_ANTERIOR, dtype: float64
nan
0   NaN
Name: VENTOSAS_ANTERIOR, dtype: float64
nan
0   NaN
Name: FORCEPS_ANTERIOR, dtype: float64
nan
0   NaN
Name: CESARIANAS_ANTERIOR, dtype: float64
nan
0    ESPONTANEA
Name: TIPO_GRAVIDEZ, dtype: object
ESPONTANEA
0    S
Name: VIGIADA, dtype: object
S
0    NaN
Name: VIGIADA_PARICULAR, dtype: object
nan
0    S
Name: VIGIADA_CENTRO_SAUDE, dtype: object
S
0    S
Name: VIGIADA_NESTE_HOSPITAL, dtype: object
S
0    NaN
Name: BACIA, dtype: object
nan
0    NaN
Name: APRESENTACAO_ADMISSAO, dtype: object
nan
0   NaN
Name: BISHOP_SCORE, dtype: float64
nan
0   NaN
Name: BISHOP_CONSISTENCIA, dtype: float64
nan
0   NaN
Name: BISHOP_DESCIDA, dtype: float64
nan
0   NaN
Name: BISHOP_DILATACAO, dtype: float64
nan
0   NaN
Name: BISHOP_EXTINCAO, dtype: float64
nan
0   NaN
Na

In [33]:
summ_stats = pd.DataFrame.from_dict(df_dict, orient="columns")

In [34]:
summ_stats = pd.concat([pd.DataFrame(col), summ_stats], axis=1)
summ_stats.rename(columns={0: "column"}, inplace=True)

In [35]:
summ_stats["latex_column"] = summ_stats["column"].apply(create_multirow_latex)

In [36]:
##add overal
silos_concat_stats = [silo for silo in silos_stats]  # keep same nr rows
    #minisilos=[silo for silo in silos] knn takes way too long
full_data_stat = pd.concat(silos_concat_stats).reset_index(drop=True)

In [37]:
for idx,silo in enumerate(silo_imputed):
       silo.to_csv("/Users/joaoalmeida/Desktop/tese_local/Obscare Giovana/imputed/silo"+str(idx+1)+".csv")

# create tables for publication

In [38]:
result={"aggregated":[]}

for col in full_data_stat.drop(columns=drop_cols).columns:
    print(col)
    
    type_col = "categorical" if col in cat_cols else "continuous"
    result["aggregated"].append(get_summary2(full_data_stat, col, type_col))
result["aggregated"].append(str(len(full_data_stat)))
summ_stats["Aggr."]=result["aggregated"]

IDADE_MATERNA
GS
0    A,RH_POSITIVO
Name: GS, dtype: object
A,RH_POSITIVO
PESO_INICIAL
PESO_ADMISSAO_INTERNAMENTO
IMC
CIGARROS
0    NaN
Name: CIGARROS, dtype: object
nan
A_PARA
0   NaN
Name: A_PARA, dtype: float64
nan
A_GESTA
0    1.0
Name: A_GESTA, dtype: float64
1.0
EUTOCITO_ANTERIOR
0   NaN
Name: EUTOCITO_ANTERIOR, dtype: float64
nan
VENTOSAS_ANTERIOR
0   NaN
Name: VENTOSAS_ANTERIOR, dtype: float64
nan
FORCEPS_ANTERIOR
0   NaN
Name: FORCEPS_ANTERIOR, dtype: float64
nan
CESARIANAS_ANTERIOR
0   NaN
Name: CESARIANAS_ANTERIOR, dtype: float64
nan
TIPO_GRAVIDEZ
0    ESPONTANEA
Name: TIPO_GRAVIDEZ, dtype: object
ESPONTANEA
VIGIADA
0    S
Name: VIGIADA, dtype: object
S
NUMERO_CONSULTAS_PRE_NATAL
VIGIADA_PARICULAR
0    NaN
Name: VIGIADA_PARICULAR, dtype: object
nan
VIGIADA_CENTRO_SAUDE
0    S
Name: VIGIADA_CENTRO_SAUDE, dtype: object
S
VIGIADA_NESTE_HOSPITAL
0    S
Name: VIGIADA_NESTE_HOSPITAL, dtype: object
S
BACIA
0    NaN
Name: BACIA, dtype: object
nan
APRESENTACAO_ADMISSAO
0    NaN
Name:

In [39]:
summ_stats["colum_tabularx"]=summ_stats["column"].apply(lambda x: x.replace("_"," "))

In [40]:
var_mapping={'IDADE_MATERNA': 'IA',
 'GS': 'GS',
 'PESO_INICIAL': 'PI',
 'PESO_ADMISSAO_INTERNAMENTO': 'PAI',
 'IMC': 'IMC',
 'CIGARROS': 'CIG',
 'ALCOOL': 'ALC',
 'ESTUPEFACIENTES': 'ESTP',
 'A_PARA': 'APARA',
 'A_GESTA': 'AGESTA',
 'EUTOCITO_ANTERIOR': 'EA',
 'VENTOSAS_ANTERIOR': 'VA',
 'FORCEPS_ANTERIOR': 'FA',
 'CESARIANAS_ANTERIOR': 'CA',
 'TIPO_GRAVIDEZ': 'TG',
 'VIGIADA': 'V',
 'NUMERO_CONSULTAS_PRE_NATAL': 'NRCPN',
 'VIGIADA_PARICULAR': 'VP',
 'VIGIADA_CENTRO_SAUDE': 'VCS',
 'VIGIADA_NESTE_HOSPITAL': 'VNH',
 'E_ALT_UT': 'EALTUT',
 'BACIA': 'B',
 'APRESENTACAO_ADMISSAO': 'AA',
 'BISHOP_SCORE': 'BS',
 'BISHOP_CONSISTENCIA': 'BC',
 'BISHOP_DESCIDA': 'BDE',
 'BISHOP_DILATACAO': 'BDI',
 'BISHOP_EXTINCAO': 'BE',
 'BISHOP_POSICAO': 'BP',
 'IDADE_GESTACIONAL_ADMISSAO': 'IGA',
 'TRAB_PARTO_ENTRADA_ESPONTANEO': 'TPEE',
 'TRAB_PARTO_ENTRADA_INDUZIDO': 'TPEI',
 'RPM': 'RPM',
 'HIPERTENSAO_CRONICA': 'HC',
 'HIPERTENSAO_GESTACIONAL': 'HG',
 'HIPERTENSAO_PRE_ECLAMPSIA': 'HPE',
 'DIABETES_GESTACIONAL': 'DG',
 'DIABETES_GESTACIONAL_DIETA': 'DGD',
 'DIABETES_GESTACIONAL_INSULINA': 'DGI',
 'DIABETES_GESTACIONAL_ANTIBIO': 'DGA',
 'DIABETES_MATERNA': 'DM',
 'DIABETES_TIPO1': 'DT1',
 'DIABETES_TIPO2': 'DT2',
 'HEMATOLOGICA': 'H',
 'RESPIRATORIA': 'R',
 'CEREBRAL': 'CE',
 'CARDIACA': 'CARD',
 'TIPO_PARTO': 'TP',
 'APRESENTACAO_NO_PARTO': 'ANP',
 'TRAB_PARTO_NO_PARTO': 'TPNP',
 'SEMANAS_GESTACAO_PARTO': 'SGP',
 'GRUPO_ROBSON': 'GR',
  "N (total)":"N (total)"}

def get_initial(x):
    #print(var_mapping[x])
    return var_mapping[x]
summ_stats["column_initial"]=summ_stats["column"].apply(lambda x: get_initial(x))

In [41]:
sum(pd.DataFrame(var_mapping.items())[1].value_counts()>1)

0

In [42]:
#column_formatstr
with open("my_table_new.tex", "w") as f:
    print(len(summ_stats.index))
    f.write(summ_stats[["column_initial","silo1","silo2","silo3","silo4","silo5","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 1","Silo 2","Silo 3","Silo 4","Silo 5","Agrr."]))


37


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_62992/1402155059.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(summ_stats[["column_initial","silo1","silo2","silo3","silo4","silo5","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 1","Silo 2","Silo 3","Silo 4","Silo 5","Agrr."]))


In [43]:
#column_formatstr
with open("my_table_check.tex", "w") as f:
    print(len(summ_stats.index))
    f.write(summ_stats[["column","silo1","silo2","silo3","silo4","silo5","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 1","Silo 2","Silo 3","Silo 4","Silo 5","Agrr."]))


37


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_62992/3536436980.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(summ_stats[["column","silo1","silo2","silo3","silo4","silo5","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 1","Silo 2","Silo 3","Silo 4","Silo 5","Agrr."]))


In [44]:
with open("my_table_new2.tex", "w") as f:
    f.write(summ_stats[["column_initial","silo6","silo7","silo8","silo9","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 6","Silo 7","Silo 8","Silo 9","Agrr."]))


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_62992/824786656.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(summ_stats[["column_initial","silo6","silo7","silo8","silo9","Aggr."]].to_latex(column_format="|l|l|l|l|l|l|",index=False,escape=False,header=["Column","Silo 6","Silo 7","Silo 8","Silo 9","Agrr."]))


## ANNEX 1

In [45]:
english_descr=pd.Series(["Mother Age","Blood Group","Initial weight","Weight on admission","BMI","Smoker during pregnancy","Number of previous pregnancies",
                        "","previous eutocitos","previous ventosas","Previous forceps","Previous c-sections","Pregnancy type","surveillance ",
                        "Number of consultations","Hospital surveillance","Private Practice surveillance ",
                        "surveillance  Primary Care","surveillance in this hospital","Pelvis Adequacy","Presentation on admission",
                        "Bishop score","Bishop Consistency","Bishop descent","Bishop dilatation","bishop extinction",
                        "bishop position","number of week admission","","","RPM","gestation diabetes","birth type","presentation of birth","",
                        "pregnancy weeks on birth","Robson Group"])
summ_stats["description"]=english_descr

In [46]:
with open("data_dict.tex", "w") as f:
    f.write(summ_stats[["column_initial","column","description"]].to_latex(column_format="|l|l|l|",index=False,escape=True,header=["Acronym","original Name","Description"]))


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_62992/1252489671.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(summ_stats[["column_initial","column","description"]].to_latex(column_format="|l|l|l|",index=False,escape=True,header=["Acronym","original Name","Description"]))


## analysis

In [47]:
silo_imputed[0]["BACIA"].value_counts()

2    7671
0     362
3       4
1       2
Name: BACIA, dtype: int64

In [48]:
silo1["BACIA"].value_counts(dropna=False)

NaN           7671
A              362
INADEQUADA       4
L                2
Name: BACIA, dtype: int64

In [49]:
silo_imputed[4]["BACIA"].value_counts()

2    14752
0     3387
1       26
3       12
Name: BACIA, dtype: int64

In [50]:
silo5["BACIA"].value_counts(dropna=False)

NaN           14752
A              3387
L                26
INADEQUADA       12
Name: BACIA, dtype: int64

In [51]:
silo_imputed[1]["BACIA"].value_counts()

2    6653
0    1885
1      24
3       4
Name: BACIA, dtype: int64

In [52]:
silo2["BACIA"].value_counts(dropna=False)

NaN           6653
A             1885
L               24
INADEQUADA       4
Name: BACIA, dtype: int64

In [53]:
silo_imputed[2]["BACIA"].value_counts()

2    4497
0     477
1      11
3       4
Name: BACIA, dtype: int64

In [54]:
silo3["BACIA"].value_counts(dropna=False)

NaN           4497
A              477
L               11
INADEQUADA       4
Name: BACIA, dtype: int64

In [55]:
silo_imputed[3]["BACIA"].value_counts()

2    2290
0      72
1       2
Name: BACIA, dtype: int64

In [56]:
silo4["BACIA"].value_counts(dropna=False)

NaN    2290
A        72
L         2
Name: BACIA, dtype: int64

In [57]:
for idx,silo in enumerate(silo_imputed):
    target="TRAB_PARTO_NO_PARTO"
    print("--------------"*4,idx+1,"--------------"*4)
    print(silo[target].value_counts(dropna=False,normalize=True))
    print(eval("silo"+str(idx+1))[target].value_counts(dropna=False,normalize=True))
    print(len(silo[target].value_counts(dropna=False,normalize=True)),len(eval("silo"+str(idx+1))[target].value_counts(dropna=False,normalize=True)))

-------------------------------------------------------- 1 --------------------------------------------------------
4     0.526434
2     0.168056
0     0.165443
10    0.073143
8     0.028859
3     0.028611
1     0.007712
7     0.000995
5     0.000373
6     0.000373
Name: TRAB_PARTO_NO_PARTO, dtype: float64
Espontâneo                            0.526434
Indução com misoprostol               0.168056
Ausência de trabalho de parto         0.165443
Indução                               0.073143
Indução ocitócica                     0.028859
Indução com dinoprostona              0.028611
Indução com misoprostol+ocitocina     0.007712
Indução com dinoprostona+ocitocina    0.000995
Indução mecânica                      0.000373
Indução mecânica+ocitocina            0.000373
Name: TRAB_PARTO_NO_PARTO, dtype: float64
10 10
-------------------------------------------------------- 2 --------------------------------------------------------
4     0.515993
10    0.355242
0     0.128765
Name: TRAB_PA